In [1]:
import glob
import pandas as pd

In [2]:
liste = glob.glob('C:/Users/joaka/Documents/Master Miashs/Marathon/*.csv')
for i in range(len(liste)):
    if i == 0 :
        db = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
    else:
        db_temp = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
        try:
            db_temp.drop('Libellé',axis=1, inplace=True)
        except:
            pass
        db = db.merge(db_temp, left_on='Code', right_on='Code', how='inner')

In [3]:
export = db[["Code","Libellé","Distance au plus proche généraliste libéral 2019",
            "Temps d'accès au généraliste libéral le plus proche 2019",
             "Nbre de généralistes lib. au 31/12 2019 Total",
             "Nbre de généralistes lib. au 31/12 2019 60> ",
             "Nbre de généralistes lib. au 31/12 2019 55 - 59",
             "Nbre de généralistes lib. au 31/12 2019 40 -  49",
             "Nbre de généralistes lib. au 31/12 2019 <40",
             "Pharmacie 2018", "Nbre de bénéficiaires d'actes gén. 2018",
             "Densité de population 2016","Taux d'emploi 2016",
             "Taux d'emploi 2016.1","Revenu fiscal médian 2016",
             "Population des moins de 18 ans 2016",
             "Population des 65 ans ou plus 2016",
             "Nombre de personnes sans diplôme 2016",
             "Nombre de personnes diplômées du supérieur 2016",
             "EHPA / EHPAD : capacité d'accueil 16-05-2019",
             "Nombre de lits et places en MCO 2017",
             "Nombre de lits en hospitalisation complète en MCO 2017"]]

# export['Code'] = export['Code'].str[:2]
export["Nbr de ben/nbr de gen"] = export["Nbre de bénéficiaires d'actes gén. 2018"]/export["Nbre de généralistes lib. au 31/12 2019 Total"]

export.columns = ['Code', 
                  'Name','DG19','TG19','NG19T','NG19_60>',
                  'NG19_55_59','NG19_40_49','NG19_<40',
                  'Pharm18','NBG18','DstPop16',
                  'TxEmpF16','TxEmpH16','RFM16','Pop16_<18','Pop16_>65',
                  '<Dpl16','>Dpl16','EHPAD19_c','NlMCO19',
                  'NlHC17','RBG_18-19']

C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
dep = pd.read_csv("dep/communes-departement-region.csv", sep=",", encoding="utf-8")
#dep = dep.groupby(['code_commune_INSEE']).count().reset_index()
dep['code_commune_INSEE'] = dep['code_commune_INSEE'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_postal'] = dep['code_postal'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_dep'] = dep['code_commune_INSEE'].str[:2]
dep = dep[['code_dep','code_commune_INSEE', 
     'nom_departement','nom_region','code_postal',
     'code_region','longitude','latitude']]

In [5]:
merged_data = pd.merge(export,dep, left_on= "Code", right_on ="code_commune_INSEE", how="inner")
data = pd.DataFrame(merged_data['Code'].value_counts() == 1)
liste_code_commune = data[data['Code']==True].index
merged_data_one = merged_data[merged_data['Code'].isin(liste_code_commune)]
merged_data_one['code_region'] = merged_data_one['code_region'].astype(int).astype(str)


C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
merged_data_one.index.values

array([    0,     1,     2, ..., 37899, 37900, 37901], dtype=int64)

In [7]:
#merged_data_one = merged_data_one.set_index(['code_region'])

In [8]:
merged_data_one.head(5)

,Code,Name,DG19,TG19,NG19T,NG19_60>,NG19_55_59,NG19_40_49,NG19_<40,Pharm18,...,NlHC17,RBG_18-19,code_dep,code_commune_INSEE,nom_departement,nom_region,code_postal,code_region,longitude,latitude
0,01001,L'Abergement-Clémenciat,5.0,7.0,0,0,0,0,0,0.0,...,0.0,inf,01,01001,Ain,Auvergne-Rhône-Alpes,01400,84,4.926114,46.153426
1,01002,L'Abergement-de-Varey,4.0,6.0,0,0,0,0,0,0.0,...,0.0,inf,01,01002,Ain,Auvergne-Rhône-Alpes,01640,84,5.428017,46.009188
2,01004,Ambérieu-en-Bugey,0.0,0.0,21,4,1,5,8,6.0,...,87.0,599.047619,01,01004,Ain,Auvergne-Rhône-Alpes,01500,84,5.372926,45.960848
3,01005,Ambérieux-en-Dombes,0.0,0.0,2,1,0,0,1,1.0,...,0.0,760.500000,01,01005,Ain,Auvergne-Rhône-Alpes,01330,84,4.912273,45.996180
4,01006,Ambléon,11.0,16.0,0,0,0,0,0,0.0,...,0.0,inf,01,01006,Ain,Auvergne-Rhône-Alpes,01300,84,5.594320,45.749499


In [15]:
merged_data_one.shape

(33173, 31)

In [9]:
merged_data_one.head(5).to_dict()

{'Code': {0: '01001', 1: '01002', 2: '01004', 3: '01005', 4: '01006'},
 'Name': {0: "L'Abergement-Clémenciat",
  1: "L'Abergement-de-Varey",
  2: 'Ambérieu-en-Bugey',
  3: 'Ambérieux-en-Dombes',
  4: 'Ambléon'},
 'DG19': {0: 5.0, 1: 4.0, 2: 0.0, 3: 0.0, 4: 11.0},
 'TG19': {0: 7.0, 1: 6.0, 2: 0.0, 3: 0.0, 4: 16.0},
 'NG19T': {0: 0, 1: 0, 2: 21, 3: 2, 4: 0},
 'NG19_60>': {0: 0, 1: 0, 2: 4, 3: 1, 4: 0},
 'NG19_55_59': {0: 0, 1: 0, 2: 1, 3: 0, 4: 0},
 'NG19_40_49': {0: 0, 1: 0, 2: 5, 3: 0, 4: 0},
 'NG19_<40': {0: 0, 1: 0, 2: 8, 3: 1, 4: 0},
 'Pharm18': {0: 0.0, 1: 0.0, 2: 6.0, 3: 1.0, 4: 0.0},
 'NBG18': {0: 648.0, 1: 219.0, 2: 12580.0, 3: 1521.0, 4: 66.0},
 'DstPop16': {0: 49.0, 1: 26.6, 2: 575.0, 3: 104.1, 4: 18.2},
 'TxEmpF16': {0: 69.3, 1: 71.6, 2: 57.1, 3: 70.9, 4: 73.3},
 'TxEmpH16': {0: 78.7, 1: 85.7, 2: 68.2, 3: 77.8, 4: 66.7},
 'RFM16': {0: 23440.0, 1: 24214.0, 2: 19450.0, 3: 23280.0, 4: nan},
 'Pop16_<18': {0: 194.0, 1: 62.0, 2: 3369.0, 3: 403.0, 4: 15.0},
 'Pop16_>65': {0: 143.0,

In [11]:
retro_dictify(merged_data_one.head(5))

['01001' "L'Abergement-Clémenciat" 5.0 7.0 0 0 0 0 0 0.0 648.0 49.0 69.3
 78.7 23440.0 194.0 143.0 131.0 141.0 0 0.0 0.0 inf '01' '01001' 'Ain'
 'Auvergne-Rhône-Alpes' '01400' '84' 4.92611354223 46.1534255214]
['01002' "L'Abergement-de-Varey" 4.0 6.0 0 0 0 0 0 0.0 219.0 26.6 71.6
 85.7 24214.0 62.0 44.0 29.0 61.0 0 0.0 0.0 inf '01' '01002' 'Ain'
 'Auvergne-Rhône-Alpes' '01640' '84' 5.42801696363 46.009187877600006]
['01004' 'Ambérieu-en-Bugey' 0.0 0.0 21 4 1 5 8 6.0 12580.0 575.0 57.1
 68.2 19450.0 3369.0 2322.0 2888.0 2588.0 144 111.0 87.0 599.047619047619
 '01' '01004' 'Ain' 'Auvergne-Rhône-Alpes' '01500' '84' 5.3729257777
 45.960847511400004]
['01005' 'Ambérieux-en-Dombes' 0.0 0.0 2 1 0 0 1 1.0 1521.0 104.1 70.9
 77.8 23280.0 403.0 257.0 344.0 289.0 0 0.0 0.0 760.5 '01' '01005' 'Ain'
 'Auvergne-Rhône-Alpes' '01330' '84' 4.91227250796 45.9961799872]
['01006' 'Ambléon' 11.0 16.0 0 0 0 0 0 0.0 66.0 18.2 73.3 66.7 nan 15.0
 26.0 21.0 13.0 0 0.0 0.0 inf '01' '01006' 'Ain' 'Auvergne-Rhône

{}

In [16]:
test = {"dsf":"sdfsdf"}

In [19]:
test['dsf']

'sdfsdf'

In [18]:
"dsf" in test.keys()

True

In [13]:
def dictionnary_python(df):
    d = {}
    for row in df.values:
        if row[28] in d.keys():
            if row[23] in d[row['28']]:
                
        else:
            d[row[28]]={}
    return d

SyntaxError: invalid syntax (<ipython-input-13-7509a045ff57>, line 4)